## **Building an End-to-End RAG Chain**

### **Steps**

> **Step 1: Initialize an Embedding Model**  
> **Step 2: Initialize the Chroma DB Connection**  
> **Step 3: Create a Retriever Object**   
> **Step 4: Initialize a Chat Prompt Template**  
> **Step 5: Initialize a Generator (i.e. Chat Model)**  
> **Step 6: Initialize a Output Parser**   
> **Step 7: Define a RAG Chain**  
> **Step 8: Invoke the Chain**

### **Step 1: Initialize an Embedding Model**

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### **Step 2: Initialize the Chroma DB Connection**

In [3]:
from langchain_chroma import Chroma

db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db")

### **Step 3: Create a Retriever Object**

In [4]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 3})

### **Step 4: Initialize a Chat Prompt Template**

In [5]:
from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don't justify your answers.
Don't give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

### **Step 5: Initialize a Generator (i.e. Chat Model)**

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    google_api_key=api_key,
    model="gemini-1.5-pro",
    temperature=1
)

### **Step 6: Initialize a Output Parser** 

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

### **Step 7: Define a RAG Chain**

In [11]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt_template | chat_model | output_parser

### **Step 8: Invoke the Chain**

In [12]:
query = "Who is Rachem?"

rag_chain.invoke(query)

'"Rachem" is a misreading of "Rachel".\n'

In [14]:
query = "Who is Rachel?"

print(rag_chain.invoke(query))

Rachel is someone described as funny, sweet, amazing, adorable, and sexy. She is a waitress and is dating someone who feels good being with her. She also has a friend named Pheebs.  Someone (possibly a different person) considers her spoiled, ditzy, and too into her looks.



In [15]:
query = "What is the List comparing Rachel and Julie?"

print(rag_chain.invoke(query))

The list describes Rachel as spoiled, ditzy, and too into her looks. It also mentions that she is a waitress, contrasting with Julie, who is a paleontologist and shares common interests with the speaker.

